# Comments and results

In the first task I calculated PDP plot (also called PDP profile). Basically just the definition is used to obtain Monte Carlo approximation of the marginal. The result is presented on the plot. We can observe a quadratic function. It's important to note that the variables are correlated and therefore the necessary assumption for using PDP method doesn't hold.

In the second part I examined Random Forest model fit to the heart dataset. The task here is to predict probability of heart attack. I have shown results for a small sample containing four examples on the plot. We can see that the local explanations differ in some cases. Locally, from the perspective of sample 101 age decreases the probability, while in sample 228 it's increased. Similar difference occurs for samples 101/228 and variable trtbps (resting blood pressure). This is not very surprising, as we are dealing with local explanations. It's easy to imagine that for certain group of patients (with specific symptoms/characteristics) probability of heart attack may indeed be higher in younger age.

Figures number 3 and 4 show results for global explanation method PDP with Random Forest and Gradient Boosting models, respectively. It's important to note that in this case we can see more stable, less sharp curves. The explanations are very similar between models, which shows that probably both models recognize similar patterns in the data.

## Plots

In [115]:
fig1

In [124]:
fig2

In [125]:
fig3

In [126]:
fig4

# Appendix

## Task 1

In [106]:
import numpy as np
import pandas as pd
import plotly.express as px
import sklearn
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from dalex import Explainer

def model(x_1, x_2):
  return (x_1+x_2)**2

x_1 = np.random.uniform(-1, 1, 1000)
x_2 = x_1

y_changed = model(x_1, x_2.mean())

df = pd.DataFrame({"x": x_1, "y": y_changed}).sort_values(by="x")
fig1 = px.line(df, x="x", y="y")

## Task 2

### Data preparation

In [107]:
df = pd.read_csv("heart.csv")
X = df.iloc[:, :13]
y = df.iloc[:, 13]

In [108]:
model = RandomForestClassifier().fit(X, y)

### Ceteris Paribus

In [109]:
ind_to_explain = [101, 228, 100, 223]
X_to_explain = X.iloc[ind_to_explain, :]

In [110]:
def predict_fun(model, x):
  return model.predict_proba(x)[:, 1]

explainer = Explainer(model, X, y, predict_function=predict_fun)
profile = explainer.predict_profile(X_to_explain)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names



Preparation of a new explainer is initiated

  -> data              : 303 rows 13 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 303 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function predict_fun at 0x7fde8904d830> will be used
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.0, mean = 0.547, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.41, mean = -0.00234, max = 0.37
  -> model_info        : package sklearn

A new explainer has been created!


Calculating ceteris paribus: 100%|██████████| 13/13 [00:00<00:00, 57.65it/s]


In [120]:
fig2 = profile.plot(variables=["age", "trtbps"], show=False)
fig2.show()

### PDP

In [121]:
pdp = explainer.model_profile()
fig3 = pdp.plot(show=False)
fig3.show()

Calculating ceteris paribus: 100%|██████████| 13/13 [00:00<00:00, 43.69it/s]


In [113]:
boost_cls = GradientBoostingClassifier()
boost_cls.fit(X, y)

explainer = Explainer(boost_cls, X, y, predict_function=predict_fun)

Preparation of a new explainer is initiated

  -> data              : 303 rows 13 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 303 values
  -> model_class       : sklearn.ensemble._gb.GradientBoostingClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function predict_fun at 0x7fde8904d830> will be used
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.00366, mean = 0.544, max = 0.995
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.767, mean = 5.85e-05, max = 0.593
  -> model_info        : package sklearn

A new explainer has been created!


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names



In [123]:
pdp = explainer.model_profile()
fig4 = pdp.plot(show=False)
fig4.show()

Calculating ceteris paribus: 100%|██████████| 13/13 [00:00<00:00, 16.36it/s]
